In [1]:
import asyncio
import os
from dotenv import load_dotenv
from typing import Dict
from pydantic import BaseModel, Field
from IPython.display import display, Markdown

import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

from agents import (
    Agent,
    WebSearchTool,
    trace,
    Runner,
    gen_trace_id,
    function_tool,
)
from agents.model_settings import ModelSettings

In [5]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

- `WebSearchTool`: lets agent search the web
- `FileSearchTool`: allows retrieving information from your OpenAI Vector Store
- `ComputerTool`: allows automating computer use tasks like taking screenshots, clicking

**Note**: Check [OpenAI Hosted Tools](https://platform.openai.com/docs/guides/agents/hosted-tools) and [Pricing](https://platform.openai.com/docs/pricing) for more information

In [16]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"), # Required to enable tools
)

In [17]:
message = "Latest AI Agent frameworks in 2025"

with trace("Web Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of June 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent systems:

- **AutoGen**: Developed by Microsoft, AutoGen is an open-source framework designed for creating multi-agent AI applications. Its architecture comprises three layers: Core, AgentChat, and Extensions, facilitating scalable and distributed agent networks. AutoGen also provides tools like AutoGen Bench for performance assessment and AutoGen Studio for no-code agent development. ([ibm.com](https://www.ibm.com/think/insights/top-ai-agent-frameworks?utm_source=openai))

- **LangChain**: LangChain enables developers to build complex AI workflows by chaining prompts, memory, and tools into coherent pipelines. It supports multiple large language models (LLMs) and APIs, making it suitable for applications like conversational agents and retrieval-augmented systems. ([lekha-bhan88.medium.com](https://lekha-bhan88.medium.com/top-5-agentic-ai-frameworks-to-watch-in-2025-9d51b2b652c0?utm_source=openai))

- **LangGraph**: LangGraph utilizes graph-based architectures to manage stateful AI workflows, ideal for orchestrating complex multi-agent systems. It offers a visual approach to managing agent interactions and dependencies, beneficial for industries requiring precise task execution, such as healthcare and supply chain management. ([blog.mechcloud.io](https://blog.mechcloud.io/top-5-ai-agent-frameworks-in-2025?utm_source=openai))

- **CrewAI**: CrewAI focuses on multi-agent collaboration, treating agents as a "crew" of "workers" to promote teamwork in AI systems. Its role-based architecture is particularly useful for applications demanding human-AI or multi-agent cooperation, such as virtual assistants and personalized learning platforms. ([blog.mechcloud.io](https://blog.mechcloud.io/top-5-ai-agent-frameworks-in-2025?utm_source=openai))

- **Eliza**: Eliza is an open-source, Web3-friendly AI agent framework that integrates seamlessly with blockchain applications. It allows developers to create AI agents capable of interacting with smart contracts and blockchain data, expanding the potential of decentralized applications. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

- **AutoAgent**: AutoAgent is a fully automated, zero-code framework enabling users to create and deploy LLM agents through natural language alone. It comprises components like Agentic System Utilities, LLM-powered Actionable Engine, Self-Managing File System, and Self-Play Agent Customization module, facilitating efficient agent development without coding requirements. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

- **AgentLite**: AgentLite is a lightweight library designed to simplify the creation and evaluation of task-oriented LLM agent systems. It offers a user-friendly platform for innovating LLM agent reasoning, architectures, and applications, enhancing the development of multi-agent systems. ([arxiv.org](https://arxiv.org/abs/2402.15538?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent development, catering to diverse applications and user expertise levels. 

In [18]:
HOW_MANY_SEARCHES = 2

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"

class WebSearchItem(BaseModel):
    reason: str = Field(
        description="Your reasoning for why this search is important to the query."
    )
    query: str = Field(
        description="The search term to use for the web search."
    )


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(
        description="A list of web searches to perform to best answer the query."
    )

# Planner Agent: Taking a query and coming up with a handful of searches that it should run
# based on that query, in order to do some deep research

planner_agent = Agent(
    name="Planner Agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan, # Structured output
)

In [19]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find an overview of current AI agent frameworks developed or popularized in 2025.', query='AI agent frameworks 2025'), WebSearchItem(reason='To gather insights on specific technologies, companies, and advancements in AI agent frameworks within the year 2025.', query='latest AI technologies 2025')]


In [20]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("anh.leduy04@hcmut.edu.vn")
    to_email = To("duyanhlucas302@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [21]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ecbdab0>, strict_json_schema=True)

In [22]:
# Email agent
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [23]:
# Report writer agent
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(
        description="A short 2-3 sentence summary of the findings."
    )

    markdown_report: str = Field(
        description="The final report"
    )

    follow_up_questions: list[str] = Field(
        description="Suggested topics to research further"
    )


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

The next 3 functions will plan and execute the search, using `planner_agent` and `search_agent`

In [ ]:
async def plan_searches(query: str):
    """Use the planner_agent to plan which searches to run for the query"""
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each item in the search plan"""
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """Use the search agent to run a web search for each item in the search plan"""
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

The next 2 functions write a report (`writer_agent`) and send email (`email_agent`)

In [27]:
async def write_report(query: str, search_results: list[str]):
    """Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """Use the email agent to send an email with the report"""
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

Let's go

In [28]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Done!")

Starting research...
Planning searches...
Will perform 2 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Done!
